<a href="https://colab.research.google.com/github/ederson-hub/Dio-Machine-Learning/blob/main/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install tensorflow opencv-python numpy

In [2]:
import os

os.makedirs('data/train', exist_ok=True)
os.makedirs('data/val', exist_ok=True)
os.makedirs('data/test', exist_ok=True)
os.makedirs('haarcascades', exist_ok=True)

In [ ]:
!wget https://github.com/opencv/opencv/raw/4.x/data/haarcascades/haarcascade_frontalface_default.xml -P haarcascades/

In [16]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yasserh/avengers-faces-dataset")

print("Path to dataset files:", path)

100%|██████████| 89.6M/89.6M [00:00<00:00, 177MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/yasserh/avengers-faces-dataset/versions/1


In [17]:
import cv2
import numpy as np

def detect_faces(image_path):
    """
    Detects faces in an image using the haarcascade classifier.

    Args:
        image_path: Path to the input image file.

    Returns:
        A list of bounding boxes (x, y, w, h) for the detected faces.
        Returns an empty list if no faces are detected or if there's an error
        reading the image.
    """
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Could not read image from {image_path}")
        return []

    face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')

    if face_cascade.empty():
        print("Error: Could not load face cascade classifier.")
        return []

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray_image,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
    )

    return faces

In [27]:
import cv2
import numpy as np

def preprocess_faces(image_path, bounding_boxes, target_size=(128, 128)):
    """
    Crops, resizes, converts to grayscale, and normalizes detected faces.

    Args:
        image_path: Path to the original image file.
        bounding_boxes: A list of bounding boxes (x, y, w, h) for detected faces.
        target_size: The desired size (width, height) for the preprocessed face images.

    Returns:
        A list of preprocessed face images as NumPy arrays, or an empty list
        if no bounding boxes are provided or the image cannot be read.
    """
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Could not read image from {image_path}")
        return []

    preprocessed_faces = []

    for (x, y, w, h) in bounding_boxes:

        face_crop = image[y:y+h, x:x+w]

        face_resized = cv2.resize(face_crop, target_size)

        face_gray = cv2.cvtColor(face_resized, cv2.COLOR_BGR2GRAY)

        face_normalized = face_gray.astype('float32') / 255.0

        preprocessed_faces.append(face_normalized)

    if preprocessed_faces:
        return np.array(preprocessed_faces)
    else:
        return []


In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [28]:
num_classes = 10

input_shape = (128, 128, 1)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',               metrics=['accuracy'])

model.summary()

train_dir = 'data/train'
val_dir = 'data/val'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical'
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,354 (12.61 MB)

 Trainable params: 3,305,354 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

Found 274 images belonging to 6 classes.
Found 235 images belonging to 6 classes.


In [30]:
epochs = 10
if 'train_generator' in locals() and train_generator.samples > 0:
    num_classes = train_generator.num_classes
    print(f"Number of classes found by train_generator: {num_classes}")

    if model.output_shape[-1] != num_classes:
         print(f"Model output shape ({model.output_shape[-1]}) does not match num_classes ({num_classes}). Redefining model.")
         input_shape = (128, 128, 1)

         model = Sequential([
             Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
             MaxPooling2D((2, 2)),
             Conv2D(64, (3, 3), activation='relu'),
             MaxPooling2D((2, 2)),
             Conv2D(128, (3, 3), activation='relu'),
             MaxPooling2D((2, 2)),
             Flatten(),
             Dense(128, activation='relu'),
             Dense(num_classes, activation='softmax')
         ])

         model.compile(optimizer=Adam(),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

         model.summary()
else:
    print("train_generator not found or has no samples. Cannot determine num_classes for model training.")
    steps_per_epoch = 0
    validation_steps = 0
    print("Cannot train the model: train_generator not available or empty.")

if 'train_generator' in locals() and 'val_generator' in locals() and train_generator.samples > 0 and val_generator.samples > 0:
    steps_per_epoch = train_generator.samples // train_generator.batch_size
    validation_steps = val_generator.samples // val_generator.batch_size

    if steps_per_epoch > 0 and validation_steps > 0:
        print("Starting model training...")
        history = model.fit(
            train_generator,
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            validation_data=val_generator,
            validation_steps=validation_steps
        )
        print("Model training complete.")
    else:
         print("Cannot train the model: Insufficient steps per epoch or validation steps.")
         print(f"Training samples found: {train_generator.samples}")
         print(f"Validation samples found: {val_generator.samples}")
         print(f"Steps per epoch: {steps_per_epoch}")
         print(f"Validation steps: {validation_steps}")

Number of classes found by train_generator: 6
Model output shape (10) does not match num_classes (6). Redefining model.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,838 (12.61 MB)

...

Starting model training...
Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2049 - loss: 1.8067

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.2032 - loss: 1.8002 - val_accuracy: 0.2812 - val_loss: 1.6039
Epoch 2/10
1/8 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.0938 - loss: 1.6846

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 381ms/step - accuracy: 0.0938 - loss: 1.6846 - val_accuracy: 0.2723 - val_loss: 1.5897
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.3051 - loss: 1.6216 - val_accuracy: 0.0982 - val_loss: 1.6812
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 378ms/step - accuracy: 0.1562 - loss: 1.6326 - val_accuracy: 0.2723 - val_loss: 1.6263
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.1903 - loss: 1.6297 - val_accuracy: 0.2768 - val_loss: 1.5416
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 363ms/step - accuracy: 0.1875 - loss: 1.6348 - val_accuracy: 0.2812 - val_loss: 1.5360
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 21s 3s/step - accuracy: 0.2850 - loss: 1.5734 - val_accuracy: 0.1250 - val_loss: 1.6644
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 368ms/step - accuracy: 0.3750 - loss: 1.5309 - val_accuracy: 0.2277 - val_loss: 1.5838
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.4185 - loss: 1.4884 - val_accuracy: 0.4598 - val_loss: 1.3608
Epoch 10/10

In [31]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

target_size = (128, 128)
batch_size = 32

try:
    train_generator = train_datagen.flow_from_directory(
        'data/train',
        target_size=target_size,
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='categorical'
    )

    val_generator = val_datagen.flow_from_directory(
        'data/val',
        target_size=target_size,
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='categorical'
    )

    num_classes = train_generator.num_classes
    print(f"Number of classes found: {num_classes}")

    if num_classes == 0:
        print("No classes found in training data. Cannot build or train the model.")
    else:
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(target_size[0], target_size[1], 1)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])

        model.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        model.summary()

         epochs = 10
        steps_per_epoch = train_generator.samples // train_generator.batch_size if train_generator.samples > 0 else 0
        validation_steps = val_generator.samples // val_generator.batch_size if val_generator.samples > 0 else 0

        if train_generator.samples > 0 and val_generator.samples > 0 and steps_per_epoch > 0 and validation_steps > 0:
            print(f"Found {train_generator.samples} training images belonging to {train_generator.num_classes} classes.")
            print(f"Found {val_generator.samples} validation images belonging to {val_generator.num_classes} classes.")
            print("Starting model training...")
            history = model.fit(
                train_generator,
                steps_per_epoch=steps_per_epoch,
                epochs=epochs,
                validation_data=val_generator,
                validation_steps=validation_steps
            )
            print("Model training complete.")
        else:
            print("Cannot train the model: Insufficient images or steps.")
            print(f"Training samples found: {train_generator.samples}")
            print(f"Validation samples found: {val_generator.samples}")
            print(f"Steps per epoch: {steps_per_epoch}")
            print(f"Validation steps: {validation_steps}")

except Exception as e:
    print(f"An error occurred during data loading or model training: {e}")
    print("Please ensure 'data/train' and 'data/val' directories exist and contain image files organized into class subdirectories.")


Found 274 images belonging to 6 classes.
Found 235 images belonging to 6 classes.
Number of classes found: 6


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,838 (12.61 MB)

...

Found 274 training images belonging to 6 classes.
Found 235 validation images belonging to 6 classes.
Starting model training...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.2193 - loss: 1.7423 - val_accuracy: 0.3393 - val_loss: 1.6110
Epoch 2/10
1/8 ━━━━━━━━━━━━━━━━━━━━ 6s 897ms/step - accuracy: 0.1875 - loss: 1.7396

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 511ms/step - accuracy: 0.1875 - loss: 1.7396 - val_accuracy: 0.2857 - val_loss: 1.5989
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.2341 - loss: 1.6325 - val_accuracy: 0.2321 - val_loss: 1.5801
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 411ms/step - accuracy: 0.1667 - loss: 1.5689 - val_accuracy: 0.3393 - val_loss: 1.5793
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.2416 - loss: 1.6250 - val_accuracy: 0.2991 - val_loss: 1.5231
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 594ms/step - accuracy: 0.2188 - loss: 1.5832 - val_accuracy: 0.4018 - val_loss: 1.5197
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.3357 - loss: 1.5502 - val_accuracy: 0.4955 - val_loss: 1.4669
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 627ms/step - accuracy: 0.6111 - loss: 1.4632 - val_accuracy: 0.3884 - val_loss: 1.4939
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.4482 - loss: 1.4309 - val_accuracy: 0.5000 - val_loss: 1.2295
Epoch 10/10

In [48]:
import numpy as np
import cv2 # Ensure cv2 is imported if used in detect_faces or preprocess_faces

def recognize_faces_in_image(image_path, model, class_names):
    """
    Detects faces in an image, preprocesses them, and classifies them using a trained model.

    Args:
        image_path: Path to the input image file.
        model: A trained Keras model for face classification.
        class_names: A list or dictionary mapping class indices to class names.

    Returns:
        A list of tuples, where each tuple contains the bounding box (x, y, w, h)
        and the predicted class name for each detected face. Returns an empty list
        if no faces are detected or if there's an error.
    """
    bounding_boxes = detect_faces(image_path)

    if bounding_boxes is None or len(bounding_boxes) == 0:
        print(f"No faces detected in {image_path}")
        return []
    if 'target_size' not in globals():
        print("Error: target_size is not defined. Cannot preprocess faces.")
        return []

    preprocessed_faces = preprocess_faces(image_path, bounding_boxes, target_size=target_size)

    if not isinstance(preprocessed_faces, np.ndarray) or preprocessed_faces.size == 0:
         print(f"Preprocessing failed or returned empty array for faces in {image_path}")
         return []

    if len(preprocessed_faces.shape) == 3:
        preprocessed_faces = np.expand_dims(preprocessed_faces, axis=-1)

    predictions = model.predict(preprocessed_faces)

    predicted_indices = np.argmax(predictions, axis=1)

    if isinstance(class_names, dict):
        predicted_names = [class_names.get(idx, "Unknown") for idx in predicted_indices]
    elif isinstance(class_names, list):
         predicted_names = [class_names[idx] if 0 <= idx < len(class_names) else "Unknown" for idx in predicted_indices]
    else:
        print("Warning: class_names is not a list or dictionary. Cannot map indices to names.")
        predicted_names = ["Unknown"] * len(predicted_indices)

    results = []
    for i, (x, y, w, h) in enumerate(bounding_boxes):
        results.append(((x, y, w, h), predicted_names[i]))

    return results

In [47]:
import cv2
from PIL import Image as PILImage
import numpy as np
from IPython.display import display, Image
import os
import io

try:
    if 'train_generator' in locals() and hasattr(train_generator, 'class_indices'):
         class_indices = train_generator.class_indices
         class_names = {v: k for k, v in class_indices.items()}
         print("Class names loaded successfully from train_generator for detection.")
    elif 'class_names' in globals() and isinstance(class_names, (list, dict)) and class_names:
         print("Class names loaded successfully from global variable for detection.")
    else:
         train_dir_check = 'data/train'
         if os.path.exists(train_dir_check):
             class_names = {i: d for i, d in enumerate(sorted(os.listdir(train_dir_check))) if os.path.isdir(os.path.join(train_dir_check, d))}
             if class_names:
                 print("Class names created from data/train directory structure for detection.")
             else:
                 class_names = {}
                 print("Warning: Could not determine class names from data/train directory.")
         else:
            class_names = {}
            print("Warning: Could not load class names. 'data/train' directory not found.")


    if 'model' in globals() and model is not None:
        model_available = True
        print("Model loaded successfully for detection.")
    else:
        print("Error: Model not found. Cannot perform face recognition.")
        model_available = False

except NameError:
    print("Warning: Model or train_generator not found. Face recognition functionality will be limited.")
    model_available = False
    class_names = {}
except Exception as e:
    print(f"An error occurred while trying to load model/class names: {e}")
    model_available = False
    class_names = {}


def on_train_button_clicked(b):
    with output_area:
        output_area.clear_output()
        print("Processing upload for training...")

        if not uploader.value:
            print("No file uploaded.")
            return

        uploaded_file_name = list(uploader.value.keys())[0]
        uploaded_file_content = uploader.value[uploaded_file_name]['content']

        class_name = class_name_input.value.strip()

        if not class_name:
            print("Please enter a class name for training.")
            return

        save_dir = os.path.join('data', 'train', class_name)
        os.makedirs(save_dir, exist_ok=True)

        save_path = os.path.join(save_dir, uploaded_file_name)

        try:
            with open(save_path, 'wb') as f:
                f.write(uploaded_file_content)

            print(f"File '{uploaded_file_name}' saved to '{save_dir}' for training class '{class_name}'.")
        except Exception as e:
            print(f"Error saving file: {e}")

def on_detect_button_clicked(b):
    with output_area:
        output_area.clear_output()
        print("Processing upload for detection...")

        if not uploader.value:
            print("No file uploaded.")
            return

        if not model_available:
             print("Cannot perform face recognition: Model not available. Please train the model first.")
             if not class_names:
                 print("Class names are also not available. Cannot proceed with recognition.")
             return
        uploaded_file_name = list(uploader.value.keys())[0]
        uploaded_file_content = uploader.value[uploaded_file_name]['content']
        temp_image_path = f"/tmp/uploaded_detection_image_{uploaded_file_name}"
        try:
            with open(temp_image_path, 'wb') as f:
                f.write(uploaded_file_content)
            print(f"Temporary file saved at {temp_image_path}")
        except Exception as e:
            print(f"Error saving temporary file: {e}")
            return

        try:
            print(f"Detecting and recognizing faces in {uploaded_file_name}...")
            if not class_names:
                 print("Error: Class names are not available. Cannot perform recognition.")
                 return
            global target_size
            face_recognition_results = recognize_faces_in_image(temp_image_path, model, class_names)

            if face_recognition_results:
                print(f"Detected and recognized {len(face_recognition_results)} faces:")

                img = cv2.imread(temp_image_path)

                if img is None:
                     print(f"Error: Could not read image from {temp_image_path} for drawing.")
                else:
                    for (x, y, w, h), name in face_recognition_results:
                        color = (0, 255, 0)
                        thickness = 2
                        cv2.rectangle(img, (x, y), (x+w, y+h), color, thickness)

                        font = cv2.FONT_HERSHEY_SIMPLEX
                        font_scale = 0.7
                        font_thickness = 2
                        text_color = (255, 255, 255)
                        text_position = (x, y - 10)

                        if text_position[1] < 10:
                            text_position = (x, y + h + 20)

                        cv2.putText(img, name, text_position, font, font_scale, text_color, font_thickness, cv2.LINE_AA)

                    result_image_path = f"/tmp/detection_result_{uploaded_file_name}"
                    cv2.imwrite(result_image_path, img)
                    print(f"Result image saved to {result_image_path}")

                    print("Attempting to display detection results:")
                    display(Image(filename=result_image_path))
                    print("Display command executed.")

            else:
                print("No faces detected or recognized in the uploaded image.")

        except Exception as e:
            print(f"An error occurred during face detection/recognition: {e}")
            import traceback
            traceback.print_exc()
        finally:

            if os.path.exists(temp_image_path):
                os.remove(temp_image_path)
                print(f"Removed temporary file: {temp_image_path}")

train_button.on_click(on_train_button_clicked)
detect_button.on_click(on_detect_button_clicked)

print("Please upload an image using the widget above.")
print("If uploading for training, enter the class name before clicking 'Upload for Training'.")
print("If uploading for detection, just upload the image and click 'Detect Faces'.")

Class names loaded successfully from train_generator for detection.
Model loaded successfully for detection.
Please upload an image using the widget above.
If uploading for training, enter the class name before clicking 'Upload for Training'.
If uploading for detection, just upload the image and click 'Detect Faces'.


In [45]:
import os
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import cv2

test_data_dir = 'data/test'
if not os.path.exists(test_data_dir) or not os.listdir(test_data_dir):
    print(f"Warning: Test data directory '{test_data_dir}' not found or is empty.")
    print("Please ensure test images are placed in class-specific subdirectories within this folder.")
    test_image_paths = []
    true_labels = []
else:
    test_image_paths = []
    true_labels = []
    for class_name in os.listdir(test_data_dir):
        class_dir = os.path.join(test_data_dir, class_name)
        if os.path.isdir(class_dir):
            for image_name in os.listdir(class_dir):
                image_path = os.path.join(class_dir, image_name)
                if os.path.isfile(image_path):
                    test_image_paths.append(image_path)
                    true_labels.append(class_name)

    print(f"Found {len(test_image_paths)} test images across {len(set(true_labels))} classes.")

try:
    if 'train_generator' in locals() and hasattr(train_generator, 'class_indices'):
         class_indices = train_generator.class_indices
         class_names_eval = {v: k for k, v in class_indices.items()}
         print("Class names loaded from train_generator.")
    elif 'class_names' in globals() and isinstance(class_names, (list, dict)):
         class_names_eval = class_names
         print("Class names loaded from global variable.")
    else:
         print("Error: Could not load class names. Cannot perform evaluation.")
         class_names_eval = {}

    if 'model' in globals() and model is not None:
        evaluation_model = model
        model_available_for_eval = True
        print("Model loaded successfully for evaluation.")
    else:
        print("Error: Model not found. Cannot perform evaluation.")
        evaluation_model = None
        model_available_for_eval = False

except Exception as e:
    print(f"An error occurred while trying to load model/class names for evaluation: {e}")
    evaluation_model = None
    class_names_eval = {}
    model_available_for_eval = False

if test_image_paths and model_available_for_eval and class_names_eval:
    predicted_labels = []
    actual_labels_for_predicted_faces = []

    print("Starting evaluation...")
    for i, image_path in enumerate(test_image_paths):
        true_label = os.path.basename(os.path.dirname(image_path))
        face_recognition_results = recognize_faces_in_image(image_path, evaluation_model, class_names_eval)

        if face_recognition_results:
            if len(face_recognition_results) > 0:
                 predicted_name = face_recognition_results[0][1]
                 predicted_labels.append(predicted_name)
                 actual_labels_for_predicted_faces.append(true_label)
        else:
            print(f"No faces detected in {image_path}. Skipping for evaluation metrics.")

    if actual_labels_for_predicted_faces:
        print("\n--- Evaluation Results ---")

        all_labels = sorted(list(set(actual_labels_for_predicted_faces + predicted_labels)))
        accuracy = accuracy_score(actual_labels_for_predicted_faces, predicted_labels)
        print(f"Accuracy: {accuracy:.4f}")
        try:
            report = classification_report(actual_labels_for_predicted_faces, predicted_labels, labels=all_labels, zero_division=0)
            print("\nClassification Report:")
            print(report)
        except ValueError as e:
             print(f"Could not generate classification report: {e}")
             print("This might happen if there's a mismatch in labels or insufficient samples.")


        print("\n--- Analysis and Potential Improvements ---")
        print("Based on the accuracy and classification report:")
        print("1. Analyze the report: Look at precision, recall, and F1-score for each class.")
        print("   - Low precision for a class means the model often predicts that class incorrectly.")
        print("   - Low recall means the model often fails to detect instances of that class.")
        print("   - The F1-score is a balance between precision and recall.")
        print("2. Review False Positives and False Negatives: If possible, visually inspect images where the model made incorrect predictions.")
        print("   - False Positives: The model predicted a class, but it was incorrect.")
        print("   - False Negatives: The model failed to predict the correct class (either missed detection or wrong classification).")
        print("\nPotential strategies for refinement and improvement:")
        print("1. Data Augmentation: Increase the diversity of the training data (e.g., more angles, lighting conditions, expressions).")
        print("2. More Training Data: A larger dataset generally leads to better generalization.")
        print("3. Hyperparameter Tuning: Experiment with learning rate, batch size, number of epochs, etc.")
        print("4. Model Architecture: Try a deeper or different CNN architecture (e.g., transfer learning with pre-trained models like VGG-Face, ResNet).")
        print("5. Face Detection Parameters: Adjust `scaleFactor`, `minNeighbors`, `minSize` in `detect_faces` to improve detection rate.")
        print("6. Preprocessing: Experiment with different preprocessing steps (e.g., histogram equalization, different normalization).")
        print("7. Loss Function: Explore different loss functions if appropriate.")
        print("8. Handle Multiple Faces: Refine the evaluation logic to correctly assess performance when multiple faces are present in an image.")
        print("9. Error Analysis: Focus on improving performance for classes with low scores.")

    else:
        print("No faces were successfully processed for evaluation. Cannot calculate metrics.")

else:
    print("\nEvaluation skipped due to missing test data, model, or class names.")


Found 274 test images across 5 classes.
Class names loaded from train_generator.
Model loaded successfully for evaluation.
Starting evaluation...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━

In [46]:
import ipywidgets as widgets
from IPython.display import display, Image
import os
import io
from PIL import Image as PILImage # Use PIL to handle image byte data

# Create a file upload widget
uploader = widgets.FileUpload(
    accept='image/*',  # Accept only image files
    multiple=False     # Allow only single file upload for simplicity
)

# Create buttons for actions
train_button = widgets.Button(description="Upload for Training")
detect_button = widgets.Button(description="Detect Faces")

# Create an output widget to display results
output_area = widgets.Output()

# Create a text input for the class name during training upload
class_name_input = widgets.Text(
    value='',
    placeholder='Enter class name (e.g., "John_Doe")',
    description='Class Name:',
    disabled=False
)

# Arrange the widgets in a layout
upload_box = widgets.VBox([
    widgets.Label("Upload an image:"),
    uploader,
    class_name_input,
    widgets.HBox([train_button, detect_button]),
    output_area
])

# Display the layout
display(upload_box)

# Placeholder functions for event handlers (will be implemented in the next step)
def on_train_button_clicked(b):
    with output_area:
        print("Train button clicked. Processing upload...")
        # Implementation will go here

def on_detect_button_clicked(b):
    with output_area:
        print("Detect button clicked. Processing upload...")
        # Implementation will go here

# Link buttons to placeholder handlers
train_button.on_click(on_train_button_clicked)
detect_button.on_click(on_detect_button_clicked)

print("Please upload an image using the widget above.")
print("If uploading for training, enter the class name before clicking 'Upload for Training'.")
print("If uploading for detection, just upload the image and click 'Detect Faces'.")

Please upload an image using the widget above.
If uploading for training, enter the class name before clicking 'Upload for Training'.
If uploading for detection, just upload the image and click 'Detect Faces'.
